In [29]:
from glob import glob
from os.path import basename, dirname, join, exists, expanduser
import pandas as pd
import numpy as np
import pickle

data_dir = '/bsuscratch/zacharykeskinen/data/uavsar/images'

In [36]:
image_fps = []
for loc_dir in glob(join(data_dir, '*')):
    for img_dir in glob(join(loc_dir, '*')):
        if not img_dir.endswith('tmp'):
            for f in glob(join(img_dir, '*unw.grd.tiff')):
                csv_fp = glob(join(img_dir, '*.csv'))[0]
                df = pd.read_csv(csv_fp, index_col= 0)
                inc_fp = glob(join(img_dir, '*.inc.tiff'))[0]
                d = {}
                d['fp'] = f
                d['ann'] = csv_fp
                d['inc'] = inc_fp
                d['cor'] = f.replace('unw','cor')
                d['hgt'] = f.replace('unw','hgt')
                d['pol'] = basename(f).split('_')[6][-2:]
                d['heading'] = basename(f).split('_')[1][:3]
                d['location'] = basename(loc_dir)
                d['flight1'] = pd.to_datetime(df.loc['value','start time of acquisition for pass 1'])
                d['flight2'] = pd.to_datetime(df.loc['value','start time of acquisition for pass 2'])
                image_fps.append(d)

In [28]:
with open(expanduser('~/scratch/data/uavsar/image_fps'), 'wb') as f:
    pickle.dump(image_fps, f)

In [19]:
loc_fps = [dic for dic in image_fps if dic['location'] == 'Salt Lake City, UT']
fps_2021 = [fs for fs in loc_fps if fs['flight1'] < pd.to_datetime('2020-10-01').tz_localize('UTC')]
fps_2021 = [fs for fs in fps_2021 if fs['pol']  == 'VV']